In [1]:
import torch
import torch.nn as nn
from motor_control import MotorControl
from torch.utils.data import Dataset

from os import path, listdir
import pandas as pd


In [10]:
model = MotorControl()

In [90]:
model.state_dict(torch.load('model.pt'))
print(model)

MotorControl(
  (fc1): Linear(in_features=28, out_features=16, bias=True)
  (relu1): ReLU()
  (fc2): Linear(in_features=16, out_features=42, bias=True)
  (sig): Sigmoid()
)


/Users/rileyjones/miniconda3/envs/hydra/lib/python3.11/site-packages/torch/nn/modules/module.py:1879: UserWarning: Positional args are being deprecated, use kwargs instead. Refer to https://pytorch.org/docs/master/generated/torch.nn.Module.html#torch.nn.Module.state_dict for details.
  warnings.warn(


In [20]:
class MotorDataset(Dataset):
    def __init__(self, src):
        self.data_labels = [f.replace(".in","") for f in listdir(src) if f.endswith(".in")]
        self.src = src
        
        def noise(example, noise_level=0.01):
            return example + (torch.randn(example.shape) * example * noise_level)
        self.noise = noise

    def __len__(self):
        return len(self.data_labels)

    def __getitem__(self, idx):
        input = pd.read_csv(path.join(self.src, self.data_labels[idx] + ".in"), header=None)
        target = pd.read_csv(path.join(self.src, self.data_labels[idx] + ".out"), header=None)
        
        input = self.noise(torch.Tensor(input.values), noise_level=0.1)
        target = self.noise(torch.Tensor(target.values))

        return input, target

ds  = MotorDataset('data/')

In [17]:
import torch.optim as optim
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

def train():
    model.train()
    for i in range(len(ds)):
        input, target = ds[i]
        output = model(input)
        loss = criterion(output, target)
        model.zero_grad()
        loss.backward()
        optimizer.step()

    return loss.item()

def test():
    model.eval()
    with torch.no_grad():
        for i in range(len(ds)):
            input, target = ds[i]
            output = model(input)
            loss = criterion(output, target)
    print("Final " + str(loss.item()))

def train_loop(epochs=100):
    for epoch in range(epochs):
        loss = train()
        print(f"Epoch {epoch} loss: {loss}")

In [21]:
train_loop(epochs=1000)
test()

Epoch 0 loss: 0.0014346848474815488
Epoch 1 loss: 0.0013709759805351496
Epoch 2 loss: 0.001811377122066915
Epoch 3 loss: 0.0015289817238226533
Epoch 4 loss: 0.00149579590652138
Epoch 5 loss: 0.0015932630049064755
Epoch 6 loss: 0.0026463817339390516
Epoch 7 loss: 0.0015759262023493648
Epoch 8 loss: 0.003380107693374157
Epoch 9 loss: 0.001188168884254992
Epoch 10 loss: 0.0016857098089531064
Epoch 11 loss: 0.0012232607696205378
Epoch 12 loss: 0.001979213673621416
Epoch 13 loss: 0.0014127760659903288
Epoch 14 loss: 0.002458044094964862
Epoch 15 loss: 0.001282877055928111
Epoch 16 loss: 0.0025007715448737144
Epoch 17 loss: 0.0013766629854217172
Epoch 18 loss: 0.0019025389337912202
Epoch 19 loss: 0.0026672091335058212
Epoch 20 loss: 0.0012262157397344708
Epoch 21 loss: 0.002334668068215251
Epoch 22 loss: 0.001517471973784268
Epoch 23 loss: 0.0018985969945788383
Epoch 24 loss: 0.0014040408423170447
Epoch 25 loss: 0.001330991042777896
Epoch 26 loss: 0.0015342987608164549
Epoch 27 loss: 0.00186

In [23]:
torch.save(model.state_dict(), 'model.pt')